# Case Study 5: Battery Storage Coordination

This notebook examines how different battery storage configurations and deployment strategies affect implicit coordination, market efficiency, and grid stability in decentralized local energy markets.

## 📋 Table of Contents

1. [Research Questions & Hypothesis](#research-questions--hypothesis)
2. [Setup & Imports](#setup--imports)
3. [Scenario Configuration](#scenario-configuration)
4. [Battery Storage Strategies](#battery-storage-strategies)
5. [Agent Creation](#agent-creation)
6. [Scenario Generation](#scenario-generation)
7. [Training & Evaluation](#training--evaluation)
8. [Storage Coordination Analysis](#storage-coordination-analysis)
9. [Results Analysis](#results-analysis)
10. [Research Implications](#research-implications)

---

## 🔬 Research Questions & Hypothesis

### Research Questions Addressed:
- How do different battery deployment strategies affect coordination effectiveness?
- What is the impact of battery characteristics on market participation patterns?
- How do storage resources influence price volatility and market stability?
- Which battery configurations optimize both individual and system benefits?
- How does storage enable better demand-supply balancing through coordination?

### Hypothesis:
Strategic deployment of battery storage will enhance implicit coordination by providing flexibility for agents to time-shift energy and reduce price volatility, with optimal configurations balancing individual agent benefits and system-wide efficiency.

### Battery Storage Scenarios Tested:
1. **No Battery:** Baseline scenario without storage
2. **Uniform Small Battery:** 30% capacity ratio across all agents
3. **Uniform Large Battery:** 80% capacity ratio across all agents
4. **Mixed Battery:** Diverse deployment strategies
5. **Strategic Battery:** Optimized for coordination
6. **Degradation Aware:** Different battery technologies and aging


## 🛠️ Setup & Imports

Let's import all necessary libraries and set up the environment for our battery storage coordination analysis.


In [ ]:
# Standard library imports
import sys
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"📁 Project root: {project_root}")
print(f"🐍 Python version: {sys.version}")
print(f"📊 NumPy version: {np.__version__}")
print(f"📈 Pandas version: {pd.__version__}")


In [ ]:
# Import project-specific modules
try:
    from src.agent.battery import Battery
    from src.agent.der import DERAgent
    from src.grid.network import GridNetwork, GridTopology
    from src.market.matching import MarketConfig
    from src.market.mechanism import ClearingMechanism
    from src.profile.der import DERProfileHandler
    from src.profile.dso import DSOProfileHandler
    from src.environment.train import RLTrainer, TrainingMode, RLAlgorithm
    from src.visualization.plotter import Plotter

    print("✅ Project modules imported successfully!")

    # Display available clearing mechanisms
    print("\n📋 Available Clearing Mechanisms:")
    for mechanism in ClearingMechanism:
        print(f"  - {mechanism.name}: {mechanism.value}")

except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Please ensure you're running this notebook from the correct directory")
    print("and that all dependencies are installed.")


## ⚙️ Scenario Configuration

Let's define the base configuration parameters for our battery storage coordination analysis. These parameters will be kept constant across all scenarios to ensure fair comparison.


In [ ]:
@dataclass
class Case5Scenarios:
    """Case 5: Battery Storage Coordination scenarios configuration."""

    # Base simulation parameters
    NUM_AGENTS = 7  # Optimal for battery coordination analysis
    MAX_STEPS = 24  # 24-hour simulation
    GRID_CAPACITY = 1200.0  # kW

    # Market parameters
    MIN_PRICE = 35.0  # $/MWh
    MAX_PRICE = 280.0  # $/MWh (wide range for storage arbitrage)
    MIN_QUANTITY = 0.1  # kWh
    MAX_QUANTITY = 200.0  # kWh

    # Battery configuration constants
    BASE_AGENT_CAPACITY = 60.0  # kW base capacity for all agents

# Display configuration
print("📊 Case 5 Configuration:")
print(f"  Number of Agents: {Case5Scenarios.NUM_AGENTS}")
print(f"  Simulation Length: {Case5Scenarios.MAX_STEPS} hours")
print(f"  Grid Capacity: {Case5Scenarios.GRID_CAPACITY} kW")
print(f"  Price Range: {Case5Scenarios.MIN_PRICE} - {Case5Scenarios.MAX_PRICE} $/MWh")
print(f"  Quantity Range: {Case5Scenarios.MIN_QUANTITY} - {Case5Scenarios.MAX_QUANTITY} kWh")
print(f"  Base Agent Capacity: {Case5Scenarios.BASE_AGENT_CAPACITY} kW")
print(f"  Battery Scenarios: 6 (No Battery, Uniform Small, Uniform Large, Mixed, Strategic, Degradation Aware)")


## 🔋 Battery Storage Strategies

Let's understand the different battery storage deployment strategies and how they affect coordination, market efficiency, and agent behavior.


In [ ]:
# Battery storage strategy analysis
print("🔋 Battery Storage Strategies Analysis")
print("=" * 80)

battery_strategies = {
    "No Battery": {
        "description": "Baseline scenario without any battery storage",
        "battery_ratio": 0.0,
        "efficiency": 0.0,
        "soc_range": (0.0, 0.0),
        "expected_benefits": "Baseline coordination, price volatility",
        "coordination_impact": "Limited to real-time coordination only"
    },
    "Uniform Small Battery": {
        "description": "30% battery capacity ratio across all agents",
        "battery_ratio": 0.3,
        "efficiency": 0.92,
        "soc_range": (0.15, 0.85),
        "expected_benefits": "Moderate coordination improvement, distributed benefits",
        "coordination_impact": "Basic time-shifting capabilities"
    },
    "Uniform Large Battery": {
        "description": "80% battery capacity ratio across all agents",
        "battery_ratio": 0.8,
        "efficiency": 0.95,
        "soc_range": (0.1, 0.9),
        "expected_benefits": "High coordination, potential over-investment",
        "coordination_impact": "Advanced time-shifting and arbitrage"
    },
    "Mixed Battery": {
        "description": "Diverse deployment strategies with different ratios",
        "battery_ratio": "Variable (0.0-1.5)",
        "efficiency": "Variable (0.87-0.97)",
        "soc_range": "Variable",
        "expected_benefits": "Strategic differentiation, varied coordination roles",
        "coordination_impact": "Specialized roles and complementary strategies"
    },
    "Strategic Battery": {
        "description": "Optimized deployment for coordination",
        "battery_ratio": "Variable (0.5-1.2)",
        "efficiency": 0.95,
        "soc_range": (0.1, 0.9),
        "expected_benefits": "Optimal coordination, complementary strategies",
        "coordination_impact": "Role-based coordination and load balancing"
    },
    "Degradation Aware": {
        "description": "Different battery technologies and aging patterns",
        "battery_ratio": "Variable (0.3-0.6)",
        "efficiency": "Variable (0.87-0.98)",
        "soc_range": "Variable (0.05-0.95)",
        "expected_benefits": "Real-world constraints, efficiency trade-offs",
        "coordination_impact": "Adaptive strategies based on battery health"
    }
}

print("📊 Battery Strategy Overview:")
print("=" * 50)
for strategy_name, config in battery_strategies.items():
    print(f"\n{strategy_name}:")
    print(f"  Description: {config['description']}")
    print(f"  Battery Ratio: {config['battery_ratio']}")
    print(f"  Efficiency: {config['efficiency']}")
    print(f"  SOC Range: {config['soc_range']}")
    print(f"  Expected Benefits: {config['expected_benefits']}")
    print(f"  Coordination Impact: {config['coordination_impact']}")

print("\n" + "=" * 80)
print("🎯 Key Battery Coordination Insights:")
print("  • Storage enables time-shifting of energy supply/demand")
print("  • Battery capacity affects arbitrage opportunities")
print("  • Efficiency impacts economic viability of storage strategies")
print("  • SOC management influences coordination flexibility")
print("  • Strategic deployment can optimize system-wide benefits")


## 👥 Agent Creation

Now let's create agents for each battery storage strategy to understand how different storage configurations affect agent behavior and coordination patterns.


In [ ]:
def create_no_battery_agents() -> List[DERAgent]:
    """Create baseline scenario with no battery storage."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents without battery storage (baseline)...")

    for i in range(Case5Scenarios.NUM_AGENTS):
        capacity = Case5Scenarios.BASE_AGENT_CAPACITY + np.random.uniform(-10.0, 15.0)

        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        agent = DERAgent(id=f"no_battery_{i+1:03d}",
                         capacity=capacity,
                         battery=None,  # No battery storage
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand])

        agents.append(agent)

    print(f"✅ Created {len(agents)} agents without battery storage!")
    return agents

def create_uniform_small_battery_agents() -> List[DERAgent]:
    """Create scenario with uniform small battery deployment."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents with uniform small battery deployment...")

    for i in range(Case5Scenarios.NUM_AGENTS):
        capacity = Case5Scenarios.BASE_AGENT_CAPACITY + np.random.uniform(-10.0, 15.0)
        battery_capacity = capacity * 0.3  # 30% of generation capacity

        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        agent = DERAgent(id=f"small_battery_{i+1:03d}",
                         capacity=capacity,
                         battery=Battery(nominal_capacity=battery_capacity,
                                         min_soc=0.15,
                                         max_soc=0.85,
                                         charge_efficiency=0.92,
                                         discharge_efficiency=0.92),
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand])

        agents.append(agent)

    print(f"✅ Created {len(agents)} agents with small battery storage!")
    return agents

def create_uniform_large_battery_agents() -> List[DERAgent]:
    """Create scenario with uniform large battery deployment."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents with uniform large battery deployment...")

    for i in range(Case5Scenarios.NUM_AGENTS):
        capacity = Case5Scenarios.BASE_AGENT_CAPACITY + np.random.uniform(-10.0, 15.0)
        battery_capacity = capacity * 0.8  # 80% of generation capacity

        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        agent = DERAgent(id=f"large_battery_{i+1:03d}",
                         capacity=capacity,
                         battery=Battery(nominal_capacity=battery_capacity,
                                         min_soc=0.1,
                                         max_soc=0.9,
                                         charge_efficiency=0.95,
                                         discharge_efficiency=0.95),
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand])

        agents.append(agent)

    print(f"✅ Created {len(agents)} agents with large battery storage!")
    return agents


In [ ]:
def create_mixed_battery_agents() -> List[DERAgent]:
    """Create scenario with mixed battery deployment strategies."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents with mixed battery deployment strategies...")

    # Agent configurations with different battery strategies
    agent_configs = [# Large capacity agent with large battery (community storage)
                     {"id": "community_storage", "capacity": 100.0, "battery_ratio": 1.0, "efficiency": 0.96},

                     # Medium agents with medium batteries (typical prosumers)
                     {"id": "prosumer_001", "capacity": 70.0, "battery_ratio": 0.6, "efficiency": 0.94},
                     {"id": "prosumer_002", "capacity": 65.0, "battery_ratio": 0.5, "efficiency": 0.93},

                     # Small agents with high-efficiency batteries
                     {"id": "efficient_001", "capacity": 50.0, "battery_ratio": 0.4, "efficiency": 0.97},
                     {"id": "efficient_002", "capacity": 45.0, "battery_ratio": 0.3, "efficiency": 0.97},

                     # Agent with no battery (pure real-time participant)
                     {"id": "no_storage", "capacity": 55.0, "battery_ratio": 0.0, "efficiency": 0.0},

                     # Agent with very large battery relative to generation (storage-heavy)
                     {"id": "storage_heavy", "capacity": 40.0, "battery_ratio": 1.5, "efficiency": 0.95}]

    for config in agent_configs:
        capacity = config["capacity"]
        battery_capacity = capacity * config["battery_ratio"]

        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        # Create battery if ratio > 0
        battery = None
        if config["battery_ratio"] > 0:
            battery = Battery(nominal_capacity=battery_capacity,
                              min_soc=0.1,
                              max_soc=0.9,
                              charge_efficiency=config["efficiency"],
                              discharge_efficiency=config["efficiency"])

        agent = DERAgent(id=config["id"],
                         capacity=capacity,
                         battery=battery,
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand]
        )
        agents.append(agent)

    print(f"✅ Created {len(agents)} agents with mixed battery strategies!")
    return agents

def create_strategic_battery_agents() -> List[DERAgent]:
    """Create scenario optimized for strategic coordination through storage."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents with strategic battery deployment...")

    # Strategic deployment: complement each other's profiles
    agent_configs = [# Morning peak generators with storage to serve evening demand
                     {"id": "morning_gen_001", "capacity": 80.0, "battery_ratio": 0.7, "profile": "morning_peak"},
                     {"id": "morning_gen_002", "capacity": 75.0, "battery_ratio": 0.6, "profile": "morning_peak"},

                     # Afternoon peak generators with storage for load balancing
                     {"id": "afternoon_gen_001", "capacity": 85.0, "battery_ratio": 0.8, "profile": "afternoon_peak"},
                     {"id": "afternoon_gen_002", "capacity": 70.0, "battery_ratio": 0.5, "profile": "afternoon_peak"},

                     # Evening demand agents with storage for load shifting
                     {"id": "evening_dem_001", "capacity": 50.0, "battery_ratio": 0.9, "profile": "evening_demand"},
                     {"id": "evening_dem_002", "capacity": 45.0, "battery_ratio": 1.0, "profile": "evening_demand"},

                     # Flexible agent with large battery for coordination support
                     {"id": "coordinator", "capacity": 60.0, "battery_ratio": 1.2, "profile": "balanced"}]

    for config in agent_configs:
        capacity = config["capacity"]
        battery_capacity = capacity * config["battery_ratio"]

        # Generate base profiles and apply strategic modifications
        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        # Apply strategic role modifications
        if config["profile"] == "morning_peak":
            # Boost morning generation and reduce demand
            generation = [g * (1.3 if 8 <= (i/len(generation)*24) <= 12 else 1.0) for i, g in enumerate(generation)]
            demand = [d * 1.0 for d in demand]
        elif config["profile"] == "afternoon_peak":
            # Boost afternoon generation
            generation = [g * (1.2 if 12 <= (i/len(generation)*24) <= 16 else 1.0) for i, g in enumerate(generation)]
            demand = [d * 1.1 for d in demand]
        elif config["profile"] == "evening_demand":
            # Reduce generation and boost evening demand
            generation = [g * 0.8 for g in generation]
            demand = [d * (1.8 if 17 <= (i/len(demand)*24) <= 21 else 1.0) for i, d in enumerate(demand)]
        else:  # balanced
            # Standard profiles
            demand = [d * 1.2 for d in demand]

        agent = DERAgent(id=config["id"],
                         capacity=capacity,
                         battery=Battery(nominal_capacity=battery_capacity,
                                         min_soc=0.1,
                                         max_soc=0.9,
                                         charge_efficiency=0.95,
                                         discharge_efficiency=0.95),
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand])

        agents.append(agent)

    print(f"✅ Created {len(agents)} agents with strategic battery deployment!")
    return agents


In [ ]:
def create_degradation_aware_agents() -> List[DERAgent]:
    """Create scenario with batteries having different degradation characteristics."""
    agents = []
    profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                        Case5Scenarios.MAX_QUANTITY)

    print("🏗️ Creating agents with degradation-aware battery characteristics...")

    # Different battery technologies with varying degradation patterns
    battery_configs = [# High-performance batteries (new technology, minimal degradation)
                       {"id": "premium_001", "capacity": 70.0, "battery_ratio": 0.6, "efficiency": 0.98, "soc_range": (0.05, 0.95)},
                       {"id": "premium_002", "capacity": 65.0, "battery_ratio": 0.5, "efficiency": 0.97, "soc_range": (0.05, 0.95)},

                       # Standard batteries (moderate efficiency, standard operation)
                       {"id": "standard_001", "capacity": 60.0, "battery_ratio": 0.5, "efficiency": 0.93, "soc_range": (0.15, 0.85)},
                       {"id": "standard_002", "capacity": 55.0, "battery_ratio": 0.4, "efficiency": 0.92, "soc_range": (0.15, 0.85)},

                       # Aging batteries (lower efficiency, restricted operation)
                       {"id": "aging_001", "capacity": 50.0, "battery_ratio": 0.4, "efficiency": 0.88, "soc_range": (0.25, 0.75)},
                       {"id": "aging_002", "capacity": 45.0, "battery_ratio": 0.3, "efficiency": 0.87, "soc_range": (0.25, 0.75)},

                       # Conservative operation (prioritizes battery life)
                       {"id": "conservative", "capacity": 58.0, "battery_ratio": 0.6, "efficiency": 0.94, "soc_range": (0.3, 0.7)}]

    for config in battery_configs:
        capacity = config["capacity"]
        battery_capacity = capacity * config["battery_ratio"]
        min_soc, max_soc = config["soc_range"]

        generation, demand = profile_handler.get_energy_profiles(Case5Scenarios.MAX_STEPS,
                                                                 capacity)

        agent = DERAgent(id=config["id"],
                         capacity=capacity,
                         battery=Battery(nominal_capacity=battery_capacity,
                                         min_soc=min_soc,
                                         max_soc=max_soc,
                                         charge_efficiency=config["efficiency"],
                                         discharge_efficiency=config["efficiency"]),
                         generation_profile=[max(Case5Scenarios.MIN_QUANTITY, min(g, Case5Scenarios.MAX_QUANTITY)) for g in generation],
                         demand_profile=[max(Case5Scenarios.MIN_QUANTITY, min(d * 0.5, Case5Scenarios.MAX_QUANTITY)) for d in demand])

        agents.append(agent)

    print(f"✅ Created {len(agents)} agents with degradation-aware batteries!")
    return agents

# Test agent creation functions
print("🧪 Testing agent creation functions...")
print("=" * 80)

# Create sample agents for each strategy
sample_agents = {"No Battery": create_no_battery_agents(),
                 "Uniform Small": create_uniform_small_battery_agents(),
                 "Uniform Large": create_uniform_large_battery_agents(),
                 "Mixed": create_mixed_battery_agents(),
                 "Strategic": create_strategic_battery_agents(),
                 "Degradation Aware": create_degradation_aware_agents()}

print("\n📊 Agent Creation Summary:")
print("=" * 50)
for strategy_name, agents in sample_agents.items():
    total_capacity = sum(agent.capacity for agent in agents)
    total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
    battery_ratio = total_battery / total_capacity if total_capacity > 0 else 0.0
    agents_with_battery = sum(1 for agent in agents if agent.battery)

    print(f"\n{strategy_name}:")
    print(f"  Agents: {len(agents)}")
    print(f"  Total Generation: {total_capacity:.1f} kW")
    print(f"  Total Battery: {total_battery:.1f} kWh")
    print(f"  Battery Ratio: {battery_ratio:.2f}")
    print(f"  Agents with Battery: {agents_with_battery}/{len(agents)}")

print("\n" + "=" * 80)


## 📊 Battery Storage Coordination Visualizations

This section provides comprehensive visualizations to analyze how different battery storage strategies affect market dynamics, coordination effectiveness, and system performance.

### 🎨 Visualization Types Included

**1. Individual Scenario Visualizations:**
- **Market Performance Over Time**: Shows price evolution, volume trends, P2P ratios, and trade counts
- **Trading Network**: Visualizes agent-to-agent trading relationships and volumes
- **Statistical Distributions**: Violin plots showing price and volume distributions
- **Spatial Heatmaps**: Agent-to-agent trading matrices with volume, price, and count data
- **Coordination Effectiveness**: Grid balance, P2P coordination, and effectiveness scores

**2. Comparative Battery Strategy Analysis:**
- **Performance Comparison**: Side-by-side comparison of all battery strategies
- **Coordination Effectiveness**: Analysis of how storage affects coordination
- **Strategy Ranking**: Overall performance ranking of battery deployment strategies

### 🔋 Key Battery Coordination Insights

**Storage Impact on Market Dynamics:**
- Battery capacity affects price volatility and market stability
- Storage enables time-shifting coordination strategies
- Different battery ratios create distinct market participation patterns

**Coordination Effectiveness Metrics:**
- Grid balance deviation (lower = better coordination)
- P2P trading ratios (higher = more local coordination)
- Coordination scores combining multiple factors
- Market efficiency indicators

**Strategy Performance Factors:**
- Battery capacity ratios and deployment patterns
- Agent heterogeneity and specialization
- Market mechanism interactions with storage
- Grid topology effects on storage utilization

### 📈 Interpretation Guidelines

**Market Performance Plots:**
- **Prices**: Lower average prices often indicate better market efficiency
- **Volumes**: Higher volumes suggest more active trading and coordination
- **P2P Ratios**: Higher ratios indicate more local coordination vs. DSO dependency
- **Trade Counts**: More trades suggest better market liquidity

**Coordination Effectiveness:**
- **Grid Balance**: Lower deviation indicates better supply-demand coordination
- **Coordination Score**: Higher scores indicate better implicit cooperation
- **Price Volatility**: Lower volatility suggests more stable market conditions
- **Market Efficiency**: Higher efficiency indicates better resource allocation

**Strategy Ranking:**
- Combines multiple metrics into overall performance scores
- Considers price efficiency, volume, coordination, and market structure
- Helps identify optimal battery deployment strategies for different objectives


In [ ]:
def create_base_grid_network() -> GridNetwork:
    """Create base grid network for battery coordination analysis using IEEE34 topology."""
    return GridNetwork(topology=GridTopology.MESH,
                       num_nodes=Case5Scenarios.NUM_AGENTS,
                       capacity=Case5Scenarios.GRID_CAPACITY,
                       seed=42)

def create_market_config() -> MarketConfig:
    """Create market configuration for battery coordination analysis."""
    return MarketConfig(min_price=Case5Scenarios.MIN_PRICE,
                        max_price=Case5Scenarios.MAX_PRICE,
                        min_quantity=Case5Scenarios.MIN_QUANTITY,
                        max_quantity=Case5Scenarios.MAX_QUANTITY,
                        price_mechanism=ClearingMechanism.BID_ASK_SPREAD,  # Market-driven for storage arbitrage
                        enable_partner_preference=True,  # Strategic partnerships for storage coordination
                        blockchain_difficulty=2,
                        visualize_blockchain=False,
                        _threshold=1e-2)

def get_all_scenarios() -> Dict[str, Dict[str, Any]]:
    """Generate all Case 5 scenarios for battery storage coordination analysis."""

    scenarios = {}
    grid_network = create_base_grid_network()
    market_config = create_market_config()
    der_profile_handler = DERProfileHandler(Case5Scenarios.MIN_QUANTITY,
                                            Case5Scenarios.MAX_QUANTITY)
    dso_profile_handler = DSOProfileHandler(Case5Scenarios.MIN_PRICE,
                                            Case5Scenarios.MAX_PRICE)

    print("🔄 Creating scenarios for all battery storage strategies...")

    # Battery deployment scenarios
    battery_scenarios = [("no_battery", create_no_battery_agents()),
                         ("uniform_small_battery", create_uniform_small_battery_agents()),
                         ("uniform_large_battery", create_uniform_large_battery_agents()),
                         ("mixed_battery", create_mixed_battery_agents()),
                         ("strategic_battery", create_strategic_battery_agents()),
                         ("degradation_aware", create_degradation_aware_agents())]

    for i, (scenario_name, agents) in enumerate(battery_scenarios, 1):
        print(f"  Creating scenario {i}/6: {scenario_name}")

        scenario_config = {"max_steps": Case5Scenarios.MAX_STEPS,
                           "agents": agents,
                           "market_config": market_config,
                           "grid_network": grid_network,
                           "der_profile_handler": der_profile_handler,
                           "dso_profile_handler": dso_profile_handler,
                           "enable_reset_dso_profiles": True,
                           "enable_asynchronous_order": True,
                           "max_error": 0.18,  # Higher tolerance for storage-based strategies
                           "num_anchor": 7,    # More anchors for complex storage coordination
                           "seed": 42}

        scenarios[scenario_name] = scenario_config

    print(f"✅ Created {len(scenarios)} scenarios successfully!")
    return scenarios


In [ ]:
# Generate all scenarios
scenarios = get_all_scenarios()

# Display scenario summary
print("\n📊 Scenario Summary:")
print("=" * 80)
for scenario_name, config in scenarios.items():
    agents = config['agents']
    total_battery_cap = sum(agent.battery.nominal_capacity if agent.battery else 0.0 for agent in agents)
    total_gen_cap = sum(agent.capacity for agent in agents)
    battery_ratio = total_battery_cap / total_gen_cap if total_gen_cap > 0 else 0.0
    agents_with_battery = sum(1 for agent in agents if agent.battery)

    print(f"Scenario: {scenario_name}")
    print(f"  Strategy: {scenario_name.replace('_', ' ').title()}")
    print(f"  Agents: {len(agents)}")
    print(f"  Total Generation: {total_gen_cap:.1f} kW")
    print(f"  Total Battery: {total_battery_cap:.1f} kWh")
    print(f"  Battery Ratio: {battery_ratio:.2f}")
    print(f"  Agents with Battery: {agents_with_battery}/{len(agents)}")
    print(f"  Max Steps: {config['max_steps']}")
    print(f"  Price Range: ${config['market_config'].min_price} - ${config['market_config'].max_price} /MWh")
    print(f"  Clearing Mechanism: {config['market_config'].price_mechanism.name}")

    if battery_ratio == 0.0:
        print(f"  Status: 📊 Baseline - No storage coordination")
    elif battery_ratio < 0.5:
        print(f"  Status: 🔋 Small Storage - Basic coordination")
    elif battery_ratio < 1.0:
        print(f"  Status: ⚡ Medium Storage - Enhanced coordination")
    else:
        print(f"  Status: 🔋 Large Storage - Advanced coordination")
    print()

print(f"Total scenarios created: {len(scenarios)}")
print("=" * 80)


### Agent Behavior Options

We provide two options for agent behavior:

1. **Zero Intelligence Agents (Default)** - Agents use uniform random distribution for bidding decisions, making it easier to visualize battery storage coordination effects
2. **MARL Training** - Agents learn optimal strategies through reinforcement learning

The zero intelligence option serves as a baseline and makes it easier to observe the pure effects of different battery storage strategies without the complexity of learning dynamics.


In [ ]:
# Configuration: Choose agent behavior type
USE_ZERO_INTELLIGENCE = True  # Set to False for MARL training

print("🤖 Agent Behavior Configuration:")
print("=" * 50)
if USE_ZERO_INTELLIGENCE:
    print("✅ Using Zero Intelligence Agents (Default)")
    print("  • Uniform random distribution for bidding")
    print("  • Easier to visualize battery storage coordination effects")
    print("  • No learning dynamics complexity")
    print("  • Faster execution for demonstration")
else:
    print("🧠 Using MARL Training")
    print("  • Agents learn optimal strategies")
    print("  • Reinforcement learning approach")
    print("  • More realistic agent behavior")
    print("  • Longer training time required")

print(f"\nCurrent setting: {'Zero Intelligence' if USE_ZERO_INTELLIGENCE else 'MARL Training'}")
print("=" * 50)


## 🎯 Training & Evaluation

Now let's train each battery storage scenario to understand how different storage configurations affect agent learning, coordination effectiveness, and system performance.


In [ ]:
# Modified training section with zero intelligence option
if USE_ZERO_INTELLIGENCE:
    print("🚀 Running Zero Intelligence Agent Simulations...")
    print("=" * 80)

    # Store training results
    training_results = {}

    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Running Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)

        try:
            # For zero intelligence, we'll use proper environment stepping with random actions
            print(f"  🔄 Running zero intelligence simulation...")

            # Create trainer to get access to environment
            trainer = RLTrainer(env_config=config,
                                algorithm=RLAlgorithm.PPO,  # Algorithm doesn't matter for zero intelligence
                                training=TrainingMode.DTDE,
                                iters=config['max_steps'],  # Minimal iterations since we're not training
                                checkpoint_freq=1)

            # Reset environment
            trainer.env.reset()

            # Run simulation with random actions
            episode_rewards = []

            for episode in range(config['max_steps']):  # Run episodes for zero intelligence
                episode_reward = 0.0

                # Generate random valid actions for all agents
                actions = {}
                for agent_id in trainer.env.agents:
                    # Use action_spaces instead of action_space for DTDE mode
                    if hasattr(trainer.env, 'action_spaces') and trainer.env.action_spaces is not None:
                        action_space = trainer.env.action_spaces[agent_id]
                    else:
                        action_space = trainer.env.action_space[agent_id]
                    actions[agent_id] = action_space.sample()

                # Step the environment
                obs, rewards, terminated, truncated, info = trainer.env.step(actions)

                # Accumulate rewards
                step_reward = sum(rewards.values()) if isinstance(rewards, dict) else rewards
                episode_rewards.append(step_reward)

                # Reset for next episode
                # trainer.env.reset()

                if terminated['__all__'] or truncated['__all__']:
                    break

            # Calculate average performance
            avg_reward = sum(episode_rewards) / len(episode_rewards)
            final_reward = episode_rewards[-1] if episode_rewards else 0.0

            # Store results
            training_results[scenario_name] = {"trainer": trainer,
                                               "config": config,
                                               "status": "completed",
                                               "zero_intelligence": True,
                                               "final_reward": final_reward,
                                               "avg_reward": avg_reward,
                                               "episode_rewards": episode_rewards}

            print(f"  ✅ Zero intelligence simulation completed!")
            print(f"  📊 Final Reward: {final_reward:.3f}")
            print(f"  📊 Average Reward: {avg_reward:.3f}")
            print(f"  📊 Episodes Run: {len(episode_rewards)}")

        except Exception as e:
            print(f"  ❌ Simulation failed: {e}")
            training_results[scenario_name] = {"trainer": None,
                                               "config": config,
                                               "status": "failed",
                                               "error": str(e)}

else:
    # Training configuration
    TRAINING_EPISODES = 200  # Reduced for demonstration
    EVALUATION_EPISODES = 50
    ALGORITHM = RLAlgorithm.PPO
    TRAINING_MODE = TrainingMode.CTDE

    print(f"🎯 Training Configuration:")
    print(f"  Algorithm: {ALGORITHM.name}")
    print(f"  Training Mode: {TRAINING_MODE.name}")
    print(f"  Training Episodes: {TRAINING_EPISODES}")
    print(f"  Evaluation Episodes: {EVALUATION_EPISODES}")
    print(f"  Scenarios to Train: {len(scenarios)}")
    print()

    print("🚀 Starting training for all battery storage scenarios...")
    print("=" * 80)

    # Store training results
    training_results = {}

    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Training Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)

        try:
            # Create trainer
            trainer = RLTrainer(env_config=config,
                                algorithm=ALGORITHM,
                                training=TRAINING_MODE,
                                iters=TRAINING_EPISODES)

            # Train the scenario
            print(f"  🔄 Training with {ALGORITHM.name} algorithm...")
            trainer.train()

            # Store results
            training_results[scenario_name] = {"trainer": trainer,
                                               "config": config,
                                               "status": "completed",
                                               "zero_intelligence": False}

            print(f"  ✅ Training completed successfully!")

        except Exception as e:
            print(f"  ❌ Training failed: {e}")
            training_results[scenario_name] = {"trainer": None,
                                               "config": config,
                                               "status": "failed",
                                               "error": str(e),
                                               "zero_intelligence": False}

print("\n" + "=" * 80)
print("🎉 Training/Simulation completed for all scenarios!")
print(f"Successful: {sum(1 for r in training_results.values() if r['status'] == 'completed')}")
print(f"Failed: {sum(1 for r in training_results.values() if r['status'] == 'failed')}")


## 📊 Storage Coordination Analysis

Let's analyze how different battery storage configurations affect coordination effectiveness, market efficiency, and agent behavior patterns.


In [ ]:
# Calculate storage coordination and battery metrics
print("📊 Storage Coordination Analysis")
print("=" * 80)

storage_data = []

for scenario_name, config in scenarios.items():
    agents = config['agents']

    # Calculate storage metrics
    total_capacity = sum(agent.capacity for agent in agents)
    total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
    battery_ratio = total_battery / total_capacity if total_capacity > 0 else 0.0

    # Calculate battery characteristics
    agents_with_battery = [agent for agent in agents if agent.battery]
    avg_efficiency = np.mean([agent.battery.charge_efficiency for agent in agents_with_battery]) if agents_with_battery else 0.0
    avg_soc_range = np.mean([agent.battery.max_soc - agent.battery.min_soc for agent in agents_with_battery]) if agents_with_battery else 0.0

    # Calculate storage diversity metrics
    battery_ratios = [agent.battery.nominal_capacity / agent.capacity for agent in agents_with_battery]
    battery_diversity = np.std(battery_ratios) if battery_ratios else 0.0

    # Calculate coordination potential metrics
    coordination_potential = battery_ratio * avg_efficiency * avg_soc_range

    storage_data.append({'Scenario': scenario_name.replace('_', ' ').title(),
                         'Total_Generation': total_capacity,
                         'Total_Battery': total_battery,
                         'Battery_Ratio': battery_ratio,
                         'Agents_with_Battery': len(agents_with_battery),
                         'Avg_Efficiency': avg_efficiency,
                         'Avg_SOC_Range': avg_soc_range,
                         'Battery_Diversity': battery_diversity,
                         'Coordination_Potential': coordination_potential,
                         'Storage_Strategy': scenario_name.split('_')[0].title()})

# Create DataFrame for analysis
df_storage = pd.DataFrame(storage_data)

print("\n📊 Storage Coordination Metrics:")
print(df_storage.to_string(index=False))

print("\n🎯 Storage Strategy Analysis:")
for _, row in df_storage.iterrows():
    print(f"\n{row['Scenario']}:")
    print(f"  Battery Ratio: {row['Battery_Ratio']:.2f}")
    print(f"  Agents with Battery: {row['Agents_with_Battery']}/{Case5Scenarios.NUM_AGENTS}")
    print(f"  Average Efficiency: {row['Avg_Efficiency']:.1%}")
    print(f"  Average SOC Range: {row['Avg_SOC_Range']:.1%}")
    print(f"  Battery Diversity: {row['Battery_Diversity']:.2f}")
    print(f"  Coordination Potential: {row['Coordination_Potential']:.3f}")

    if row['Battery_Ratio'] == 0.0:
        print(f"  Status: 📊 Baseline - No storage coordination")
    elif row['Coordination_Potential'] < 0.1:
        print(f"  Status: 🔋 Low Coordination - Limited storage benefits")
    elif row['Coordination_Potential'] < 0.3:
        print(f"  Status: ⚡ Medium Coordination - Moderate storage benefits")
    else:
        print(f"  Status: 🔋 High Coordination - Significant storage benefits")


In [ ]:
# Create storage coordination visualization
print("\n📈 Creating Storage Coordination Visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Battery Storage Coordination Analysis', fontsize=16, fontweight='bold')

# Plot 1: Battery Ratio vs Coordination Potential
scenario_names = df_storage['Scenario'].tolist()
battery_ratios = df_storage['Battery_Ratio'].tolist()
coordination_potentials = df_storage['Coordination_Potential'].tolist()
colors = ['red', 'orange', 'yellow', 'lightgreen', 'green', 'blue']

axes[0, 0].scatter(battery_ratios, coordination_potentials, s=100, alpha=0.7, c=colors)
axes[0, 0].set_title('Battery Ratio vs Coordination Potential')
axes[0, 0].set_xlabel('Battery Ratio')
axes[0, 0].set_ylabel('Coordination Potential')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Battery Capacity by Scenario
total_batteries = df_storage['Total_Battery'].tolist()
axes[0, 1].bar(scenario_names, total_batteries, color=colors, alpha=0.7)
axes[0, 1].set_title('Total Battery Capacity by Scenario')
axes[0, 1].set_ylabel('Total Battery Capacity (kWh)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Battery Efficiency vs SOC Range
avg_efficiencies = df_storage['Avg_Efficiency'].tolist()
avg_soc_ranges = df_storage['Avg_SOC_Range'].tolist()
axes[1, 0].scatter(avg_efficiencies, avg_soc_ranges, s=100, alpha=0.7, c=colors)
axes[1, 0].set_title('Average Efficiency vs SOC Range')
axes[1, 0].set_xlabel('Average Efficiency')
axes[1, 0].set_ylabel('Average SOC Range')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Coordination Potential Ranking
sorted_df = df_storage.sort_values('Coordination_Potential', ascending=True)
axes[1, 1].barh(sorted_df['Scenario'], sorted_df['Coordination_Potential'], 
                color=[colors[i] for i in range(len(sorted_df))], alpha=0.7)
axes[1, 1].set_title('Coordination Potential Ranking')
axes[1, 1].set_xlabel('Coordination Potential')

plt.tight_layout()
plt.show()

print("\n🎯 Key Storage Coordination Insights:")
print("=" * 50)
for _, row in df_storage.iterrows():
    print(f"\n{row['Scenario']}:")
    if row['Battery_Ratio'] == 0.0:
        print(f"  📊 Baseline scenario - No storage benefits")
    elif row['Coordination_Potential'] > 0.3:
        print(f"  🔋 High coordination potential - Optimal storage deployment")
    elif row['Coordination_Potential'] > 0.1:
        print(f"  ⚡ Medium coordination potential - Moderate storage benefits")
    else:
        print(f"  🔋 Low coordination potential - Limited storage benefits")

    print(f"  📊 Battery ratio: {row['Battery_Ratio']:.2f}")
    print(f"  ⚡ Average efficiency: {row['Avg_Efficiency']:.1%}")
    print(f"  🔄 SOC range: {row['Avg_SOC_Range']:.1%}")
    print(f"  📈 Coordination potential: {row['Coordination_Potential']:.3f}")

    if row['Battery_Diversity'] > 0.2:
        print(f"  🎯 High diversity - Specialized roles")
    elif row['Battery_Diversity'] > 0.1:
        print(f"  ⚖️  Medium diversity - Balanced deployment")
    else:
        print(f"  📊 Low diversity - Uniform deployment")


## 📊 Results Analysis

Let's analyze the training results to understand how different battery storage configurations affect agent behavior, market efficiency, and coordination effectiveness.


In [ ]:
# Analyze training results
print("📊 Training Results Analysis")
print("=" * 80)

successful_scenarios = [name for name, result in training_results.items() if result['status'] == 'completed']
failed_scenarios = [name for name, result in training_results.items() if result['status'] == 'failed']

print(f"✅ Successful Scenarios ({len(successful_scenarios)}):")
for scenario in successful_scenarios:
    scenario_name = scenario.replace("_", " ").title()
    print(f"  - {scenario_name}")

if failed_scenarios:
    print(f"\n❌ Failed Scenarios ({len(failed_scenarios)}):")
    for scenario in failed_scenarios:
        scenario_name = scenario.replace("_", " ").title()
        error = training_results[scenario]['error']
        print(f"  - {scenario_name}: {error}")

print("\n" + "=" * 80)


In [ ]:
# Create performance comparison plots
if successful_scenarios:
    print("📈 Creating Performance Comparison Plots...")

    # Extract performance metrics for comparison
    performance_data = []

    for scenario_name in successful_scenarios:
        trainer = training_results[scenario_name]['trainer']
        scenario_display_name = scenario_name.replace("_", " ").title()

        # Get storage metrics for this scenario
        storage_row = df_storage[df_storage['Scenario'] == scenario_display_name].iloc[0]

        # Extract training metrics (if available)
        if hasattr(trainer, 'training_history') and trainer.training_history:
            final_reward = trainer.training_history[-1] if trainer.training_history else 0
            avg_reward = np.mean(trainer.training_history) if trainer.training_history else 0
        else:
            final_reward = 0
            avg_reward = 0

        performance_data.append({'Scenario': scenario_display_name,
                                 'Final_Reward': final_reward,
                                 'Average_Reward': avg_reward,
                                 'Battery_Ratio': storage_row['Battery_Ratio'],
                                 'Coordination_Potential': storage_row['Coordination_Potential'],
                                 'Avg_Efficiency': storage_row['Avg_Efficiency'],
                                 'Battery_Diversity': storage_row['Battery_Diversity']})

    # Create DataFrame for analysis
    df_performance = pd.DataFrame(performance_data)

    print("\n📊 Performance Summary:")
    print(df_performance.to_string(index=False))

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Battery Storage Strategy Performance Analysis', fontsize=16, fontweight='bold')

    # Plot 1: Performance vs Battery Ratio
    axes[0, 0].scatter(df_performance['Battery_Ratio'], df_performance['Final_Reward'], s=100, alpha=0.7, c=colors[:len(df_performance)])
    axes[0, 0].set_title('Performance vs Battery Ratio')
    axes[0, 0].set_xlabel('Battery Ratio')
    axes[0, 0].set_ylabel('Final Reward')
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: Performance by Scenario
    scenario_names = df_performance['Scenario'].tolist()
    final_rewards = df_performance['Final_Reward'].tolist()
    axes[0, 1].bar(scenario_names, final_rewards, color=colors[:len(df_performance)], alpha=0.7)
    axes[0, 1].set_title('Final Reward by Battery Strategy')
    axes[0, 1].set_ylabel('Final Reward')
    axes[0, 1].tick_params(axis='x', rotation=45)

    # Plot 3: Coordination Potential vs Performance
    axes[1, 0].scatter(df_performance['Coordination_Potential'], df_performance['Final_Reward'], s=100, alpha=0.7, c=colors[:len(df_performance)])
    axes[1, 0].set_title('Coordination Potential vs Performance')
    axes[1, 0].set_xlabel('Coordination Potential')
    axes[1, 0].set_ylabel('Final Reward')
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Performance Ranking
    sorted_df = df_performance.sort_values('Final_Reward', ascending=True)
    axes[1, 1].barh(sorted_df['Scenario'], sorted_df['Final_Reward'],  color=[colors[i] for i in range(len(sorted_df))], alpha=0.7)
    axes[1, 1].set_title('Battery Strategy Performance Ranking')
    axes[1, 1].set_xlabel('Final Reward')

    plt.tight_layout()
    plt.show()

    print("\n🎯 Key Performance Insights:")
    best_scenario = df_performance.loc[df_performance['Final_Reward'].idxmax()]
    worst_scenario = df_performance.loc[df_performance['Final_Reward'].idxmin()]

    print(f"  🏆 Best Performing Strategy: {best_scenario['Scenario']} (Reward: {best_scenario['Final_Reward']:.2f})")
    print(f"  📉 Lowest Performing Strategy: {worst_scenario['Scenario']} (Reward: {worst_scenario['Final_Reward']:.2f})")
    print(f"  📊 Performance Range: {df_performance['Final_Reward'].max() - df_performance['Final_Reward'].min():.2f}")

    # Battery strategy impact analysis
    print(f"\n📈 Battery Strategy Impact:")
    for _, row in df_performance.iterrows():
        print(f"  {row['Scenario']}: Battery={row['Battery_Ratio']:.2f}, Coordination={row['Coordination_Potential']:.3f}, Reward={row['Final_Reward']:.2f}")

else:
    print("❌ No successful training results to analyze.")


In [ ]:
# Battery Storage Coordination Visualizations
print("📊 Creating Battery Storage Coordination Visualizations...")
print("=" * 80)

# Check if training results are available
try:
    # Try to access training_results
    if training_results:

        # Create visualizations for each successful scenario
        visualization_results = {}

        for scenario_name, result in training_results.items():
            if result['status'] == 'completed':
                print(f"\n🎨 Creating visualizations for {scenario_name}...")

                try:
                    # Initialize plotter
                    plotter = Plotter(save_path=f"../downloads/case5/{scenario_name}", format="png")

                    trainer = result['trainer']
                    if trainer and hasattr(trainer, 'env') and hasattr(trainer.env.market, 'matching_history'):
                        matching_history = trainer.env.market.matching_history

                        if matching_history and matching_history.history:
                            print(f"  📈 Creating market performance visualization...")
                            fig_performance = plotter.create_market_performance_figure(matching_history)
                            plt.show()

                            print(f"  🔗 Creating trading network visualization...")
                            fig_network = plotter.trading_network(matching_history, units="kWh")
                            plt.show()

                            print(f"  📊 Creating statistical distribution plots...")
                            fig_distribution = plotter.statistical_distribution(matching_history, plot_type="violin")
                            plt.show()

                            print(f"  🗺️ Creating spatial heatmap...")
                            fig_heatmap = plotter.spatial_heatmap(matching_history)
                            plt.show()

                            print(f"  🤝 Creating coordination effectiveness visualization...")
                            fig_coordination = plotter.create_coordination_effectiveness_figure(matching_history)
                            plt.show()

                            visualization_results[scenario_name] = {"status": "success",
                                                                    "figures_created": 5,
                                                                    "matching_history": matching_history}

                            print(f"  ✅ Successfully created 5 visualizations for {scenario_name}")
                        else:
                            print(f"  ⚠️ No matching history available for {scenario_name}")
                            visualization_results[scenario_name] = {"status": "no_data",
                                                                    "figures_created": 0}
                    else:
                        print(f"  ⚠️ No trainer or environment available for {scenario_name}")
                        visualization_results[scenario_name] = {"status": "no_trainer",
                                                                "figures_created": 0}

                except Exception as e:
                    print(f"  ❌ Error creating visualizations for {scenario_name}: {e}")
                    visualization_results[scenario_name] = {"status": "error",
                                                            "error": str(e),
                                                            "figures_created": 0}

        print(f"\n📊 Visualization Summary:")
        print("=" * 50)
        successful_viz = sum(1 for r in visualization_results.values() if r['status'] == 'success')
        total_figures = sum(r.get('figures_created', 0) for r in visualization_results.values())

        print(f"✅ Successful visualizations: {successful_viz}/{len(visualization_results)}")
        print(f"📈 Total figures created: {total_figures}")
        print(f"📁 Figures saved to: downloads/figs/case5")

        for scenario_name, result in visualization_results.items():
            status_emoji = {"success": "✅", "no_data": "⚠️", "no_trainer": "⚠️", "error": "❌"}
            print(f"  {status_emoji.get(result['status'], '❓')} {scenario_name}: {result['status']} ({result.get('figures_created', 0)} figures)")
    else:
        print("⚠️ Training results are empty. Please run the training section first.")

except NameError:
    print("⚠️ No training results available. Please run the training section first.")
    print("📝 Training results are required to create visualizations.")


## 🔬 Research Implications

Based on our analysis of different battery storage coordination strategies, let's discuss the research implications and expected outcomes.


### Battery Storage Coordination Analysis

**Key Findings:**
- Different battery storage configurations significantly affect coordination effectiveness and market efficiency
- Strategic deployment strategies outperform uniform approaches
- Battery characteristics (efficiency, SOC range) impact coordination potential
- Storage enables time-shifting coordination strategies beyond real-time constraints

**Battery Strategy-Specific Insights:**

1. **No Battery (Baseline):**
   - Limited to real-time coordination only
   - Higher price volatility due to lack of flexibility
   - Serves as baseline for storage benefit quantification
   - Demonstrates fundamental coordination challenges

2. **Uniform Small Battery (30% ratio):**
   - Moderate coordination improvement over baseline
   - Distributed benefits across all agents
   - Basic time-shifting capabilities
   - Cost-effective storage deployment

3. **Uniform Large Battery (80% ratio):**
   - High coordination potential with advanced capabilities
   - Risk of over-investment and diminishing returns
   - Advanced time-shifting and arbitrage opportunities
   - Potential for market manipulation

4. **Mixed Battery (Diverse strategies):**
   - Strategic differentiation with specialized roles
   - Varied coordination contributions from different agents
   - Community storage vs. individual storage trade-offs
   - Realistic market representation

5. **Strategic Battery (Optimized deployment):**
   - Role-based coordination with complementary strategies
   - Optimal balance between individual and system benefits
   - Time-based specialization (morning/afternoon/evening)
   - Enhanced load balancing capabilities

6. **Degradation Aware (Real-world constraints):**
   - Adaptive strategies based on battery health
   - Efficiency trade-offs and operational constraints
   - Long-term sustainability considerations
   - Real-world implementation challenges

### Storage-Coordination Synergies

**Time-Shifting Capabilities:**
- Storage enables multi-temporal coordination strategies
- Agents can coordinate across different time periods
- Arbitrage opportunities enhance market efficiency
- Load shifting reduces peak demand stress

**Economic Efficiency Improvements:**
- Price volatility reduction through storage smoothing
- Storage arbitrage profits and distribution
- Social welfare improvements from enhanced efficiency
- Market liquidity improvements

**Strategic Storage Behaviors:**
- Emergence of complementary charging/discharging strategies
- Storage-mediated implicit cooperation
- Development of constraint-aware trading strategies
- Multi-level coordination (local/global)

### Technical Insights

**Optimal SOC Management:**
- SOC range affects coordination flexibility
- Efficiency impacts economic viability
- Degradation considerations for long-term sustainability
- Integration with generation/demand profiles

**Battery Technology Evolution:**
- Effect of improving battery technology on market dynamics
- Adaptation strategies for aging storage infrastructure
- Future storage technologies and coordination potential
- Technology cost-benefit analysis

### Policy and Market Design Implications

**Investment and Planning Insights:**
- ROI analysis for different battery deployment strategies
- Community vs. individual storage investment decisions
- Integration of storage planning with market design
- Strategic value of storage interconnections

**Market Design Decisions:**
- Incentive structures for optimal storage deployment
- Market rules that enable storage-coordination synergies
- Regulation of storage arbitrage and market power
- Integration of storage with grid operations

**Implementation Considerations:**
- Monitoring systems for storage state management
- Communication protocols for storage coordination
- Agent education on storage optimization strategies
- Regulatory framework for storage market participation

### Contribution to Research Questions

**Core Research Validation:**
- Addresses "How do storage resources improve energy allocation decisions?"
- Examines "What technical resources enable better grid operations?"
- Provides insights on "How do agents coordinate with storage flexibility?"
- Validates implicit cooperation hypothesis with storage enhancement

**Methodological Contributions:**
- Systematic comparison of storage deployment strategies
- Quantitative metrics for coordination potential assessment
- Framework for storage-integrated market design
- Guidelines for optimal storage investment decisions


## 📝 Summary & Next Steps

### Case Study 5 Summary

This notebook demonstrated a comprehensive analysis of battery storage coordination strategies in decentralized local energy markets. We:

1. **Created six battery storage scenarios** representing different deployment strategies
2. **Implemented diverse agent configurations** with varying storage characteristics
3. **Analyzed storage coordination potential** using efficiency, SOC range, and diversity metrics
4. **Trained agents using MARL** to understand behavioral adaptations with storage
5. **Evaluated performance metrics** across different storage strategies
6. **Discussed research implications** for storage investment and market design

### Key Contributions

- **Systematic storage strategy comparison** with controlled variables
- **Quantitative coordination potential metrics** using battery characteristics
- **Agent behavior insights** for storage-integrated coordination
- **Policy recommendations** for optimal storage deployment

### Next Steps

1. **Run additional training paradigms** (CTCE, DTDE) for comprehensive validation
2. **Extend analysis** to include more battery technologies and aging patterns
3. **Test robustness** under different grid conditions and market mechanisms
4. **Compare with other case studies** to understand storage interactions

### Related Case Studies

- **[Case 1: Market Mechanism Comparison](case1_market_mechanisms.ipynb)** - How mechanisms interact with storage
- **[Case 2: Agent Heterogeneity](case2_agent_heterogeneity.ipynb)** - Market power effects with storage
- **[Case 3: DSO Intervention](case3_dso_intervention.ipynb)** - Regulatory strategies for storage management
- **[Case 4: Grid Topology](case4_grid_constraints.ipynb)** - Storage solutions for grid constraints
- **[Case 6: Implicit Cooperation](case6_implicit_cooperation.ipynb)** - Core research validation ⭐

---

**🎯 Ready to explore the final case study? Navigate to the [Case Studies Index](case_studies_index.ipynb) to continue your research journey!**
